# EDA 및 전처리

이 노트북에서는 다음을 수행합니다:
1. 더미 데이터 로드 및 탐색
2. 정상성 테스트 (ADF, KPSS) 수행 및 시각화
3. STL 분해 수행 및 시각화
4. 비정상 시계열에 차분 적용

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 프로젝트 루트를 경로에 추가
sys.path.insert(0, str(Path.cwd().parent))

from config import Config
from src.preprocess import (
    preprocess_pipeline,
    check_stationarity_all_items,
    decompose_all_items
)

# 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

## 1. 데이터 로드

In [ ]:
# 더미 데이터 로드
data_path = Config.DATA_RAW / "dummy_trade_data.csv"

if not data_path.exists():
    print(f"⚠️ 데이터 파일이 없습니다: {data_path}")
    print("먼저 00_dummy_data_generator.ipynb를 실행하세요.")
else:
    # 전처리 파이프라인 실행
    df_wide = preprocess_pipeline(data_path, check_quality=True)
    print(f"\n데이터 shape: {df_wide.shape}")
    print(f"날짜 범위: {df_wide.index.min()} ~ {df_wide.index.max()}")
    print(f"품목 수: {len(df_wide.columns)}")

In [ ]:
# 데이터 미리보기
df_wide.head(10)

In [ ]:
# 기본 통계
df_wide.describe()

## 2. 시계열 시각화

일부 품목의 시계열 패턴을 시각화합니다.

In [ ]:
# 처음 9개 품목의 시계열 플롯
fig, axes = plt.subplots(3, 3, figsize=(16, 10))
axes = axes.flatten()

for i, col in enumerate(df_wide.columns[:9]):
    axes[i].plot(df_wide.index, df_wide[col], linewidth=1.5)
    axes[i].set_title(f'{col}', fontsize=11, fontweight='bold')
    axes[i].set_xlabel('날짜')
    axes[i].set_ylabel('무역량')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 정상성 테스트 (ADF & KPSS)

모든 품목에 대해 ADF와 KPSS 검정을 수행합니다.

In [ ]:
# 정상성 테스트 수행
stationarity_results = check_stationarity_all_items(df_wide, significance_level=0.05)

# 결과 미리보기
print("\n정상성 테스트 결과 (처음 10개):")
stationarity_results.head(10)

In [ ]:
# 결과 요약
summary = {
    'ADF 정상성': stationarity_results['adf_is_stationary'].sum(),
    'KPSS 정상성': stationarity_results['kpss_is_stationary'].sum(),
    '양쪽 모두 정상성': stationarity_results['both_stationary'].sum(),
    '총 품목 수': len(stationarity_results)
}

print("\n=" * 50)
print("정상성 테스트 요약")
print("=" * 50)
for key, value in summary.items():
    if key != '총 품목 수':
        pct = value / summary['총 품목 수'] * 100
        print(f"{key}: {value}개 ({pct:.1f}%)")
    else:
        print(f"{key}: {value}개")
print("=" * 50)

In [ ]:
# 정상성 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ADF vs KPSS
crosstab = pd.crosstab(stationarity_results['adf_is_stationary'], 
                       stationarity_results['kpss_is_stationary'],
                       rownames=['ADF 정상성'],
                       colnames=['KPSS 정상성'])

sns.heatmap(crosstab, annot=True, fmt='d', cmap='Blues', ax=axes[0], cbar_kws={'label': '품목 수'})
axes[0].set_title('ADF vs KPSS 정상성 테스트 결과', fontsize=12, fontweight='bold')

# P-value 분포
axes[1].hist(stationarity_results['adf_p_value'], bins=20, alpha=0.6, label='ADF p-value', color='blue')
axes[1].hist(stationarity_results['kpss_p_value'], bins=20, alpha=0.6, label='KPSS p-value', color='orange')
axes[1].axvline(0.05, color='red', linestyle='--', linewidth=2, label='α=0.05')
axes[1].set_xlabel('P-value', fontsize=11)
axes[1].set_ylabel('빈도', fontsize=11)
axes[1].set_title('P-value 분포', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 비정상 품목 확인
non_stationary_items = stationarity_results[~stationarity_results['both_stationary']]['item_code'].tolist()

print(f"\n비정상 품목 수: {len(non_stationary_items)}개")
if len(non_stationary_items) > 0:
    print("\n비정상 품목 목록 (처음 20개):")
    print(non_stationary_items[:20])

## 4. STL 분해 (Seasonal-Trend-Loess)

시계열을 추세, 계절성, 잔차로 분해합니다.

In [ ]:
# STL 분해 수행
decomposition_results = decompose_all_items(df_wide, period=12, seasonal=7)

df_trend = decomposition_results['trend']
df_seasonal = decomposition_results['seasonal']
df_resid = decomposition_results['resid']

print(f"\nTrend shape: {df_trend.shape}")
print(f"Seasonal shape: {df_seasonal.shape}")
print(f"Residual shape: {df_resid.shape}")

In [ ]:
# 샘플 품목의 STL 분해 시각화
sample_items = df_wide.columns[:3]

fig, axes = plt.subplots(len(sample_items), 4, figsize=(18, 4*len(sample_items)))

for i, item in enumerate(sample_items):
    # 원본
    axes[i, 0].plot(df_wide.index, df_wide[item], color='black', linewidth=1.5)
    axes[i, 0].set_title(f'{item} - 원본', fontsize=11, fontweight='bold')
    axes[i, 0].set_ylabel('값')
    axes[i, 0].grid(True, alpha=0.3)
    
    # 추세
    axes[i, 1].plot(df_trend.index, df_trend[item], color='blue', linewidth=1.5)
    axes[i, 1].set_title(f'{item} - 추세', fontsize=11, fontweight='bold')
    axes[i, 1].set_ylabel('추세')
    axes[i, 1].grid(True, alpha=0.3)
    
    # 계절성
    axes[i, 2].plot(df_seasonal.index, df_seasonal[item], color='green', linewidth=1.5)
    axes[i, 2].set_title(f'{item} - 계절성', fontsize=11, fontweight='bold')
    axes[i, 2].set_ylabel('계절성')
    axes[i, 2].grid(True, alpha=0.3)
    
    # 잔차
    axes[i, 3].plot(df_resid.index, df_resid[item], color='red', linewidth=1.5, alpha=0.7)
    axes[i, 3].set_title(f'{item} - 잔차', fontsize=11, fontweight='bold')
    axes[i, 3].set_ylabel('잔차')
    axes[i, 3].axhline(0, color='black', linestyle='--', linewidth=0.8)
    axes[i, 3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. 차분 적용 (Differencing)

비정상 시계열에 1차 차분을 적용합니다.

In [ ]:
# 1차 차분 적용
df_diff = df_wide.diff().dropna()

print(f"차분 후 shape: {df_diff.shape}")
print(f"제거된 행 수: {len(df_wide) - len(df_diff)}")

In [ ]:
# 차분 전후 비교 (비정상 품목)
if len(non_stationary_items) > 0:
    sample_non_stationary = non_stationary_items[:3]
    
    fig, axes = plt.subplots(len(sample_non_stationary), 2, figsize=(14, 4*len(sample_non_stationary)))
    
    if len(sample_non_stationary) == 1:
        axes = axes.reshape(1, -1)
    
    for i, item in enumerate(sample_non_stationary):
        # 원본
        axes[i, 0].plot(df_wide.index, df_wide[item], linewidth=1.5, color='blue')
        axes[i, 0].set_title(f'{item} - 원본 (비정상)', fontsize=11, fontweight='bold')
        axes[i, 0].set_ylabel('값')
        axes[i, 0].grid(True, alpha=0.3)
        
        # 차분 후
        axes[i, 1].plot(df_diff.index, df_diff[item], linewidth=1.5, color='green')
        axes[i, 1].set_title(f'{item} - 1차 차분 후', fontsize=11, fontweight='bold')
        axes[i, 1].set_ylabel('차분값')
        axes[i, 1].axhline(0, color='black', linestyle='--', linewidth=0.8)
        axes[i, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("모든 품목이 정상성을 만족합니다.")

## 6. 데이터 저장

분석 결과를 저장합니다.

In [ ]:
# 정상성 테스트 결과 저장
output_path = Config.DATA_PROCESSED / "stationarity_results.csv"
stationarity_results.to_csv(output_path, index=False)
print(f"✓ 정상성 테스트 결과 저장: {output_path}")

# STL 분해 결과 저장
df_trend.to_csv(Config.DATA_PROCESSED / "decomposition_trend.csv")
df_seasonal.to_csv(Config.DATA_PROCESSED / "decomposition_seasonal.csv")
df_resid.to_csv(Config.DATA_PROCESSED / "decomposition_resid.csv")
print(f"✓ STL 분해 결과 저장")

# 차분 데이터 저장
df_diff.to_csv(Config.DATA_PROCESSED / "data_differenced.csv")
print(f"✓ 차분 데이터 저장")

print("\n✓ 모든 분석 결과가 저장되었습니다!")